# 1. Carga Dataset

In [104]:
import os
import requests

# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')

# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    url = 'https://docs.google.com/uc?export=\download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)


In [105]:
import pandas as pd 

df_inicial=pd.read_csv("data/covertype/covertype_train.csv")
df_inicial

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116198,3150,220,16,285,47,2275,200,253,187,866,Commanche,C7756,1
116199,3125,47,13,234,2,2430,224,212,120,1426,Rawah,C7745,0
116200,3166,152,11,67,0,1275,234,240,136,2404,Rawah,C7202,0
116201,3154,285,14,738,46,6012,181,239,198,1320,Rawah,C7745,1


# 2. Selección de características

Se listan las columnas del dataset

In [106]:
df_inicial.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type',
       'Cover_Type'],
      dtype='object')

Se divide el dataset en 3 dataframes diferentes, los cuales corresponderán a la información númerica, categorica y variable objetivo

In [107]:
df_inicial_numeric=df_inicial[[ 'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways','Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm','Horizontal_Distance_To_Fire_Points']]

df_inicial_categoric=df_inicial[[ 'Wilderness_Area', 'Soil_Type' ]]

target=df_inicial[ 'Cover_Type' ]
target = target.astype("category")

target.value_counts()

1    56720
0    42307
2     7228
6     4045
5     3478
4     1892
3      533
Name: Cover_Type, dtype: int64

con la ayuda de las fucniones SelectKBest y f_classif de la librería se realizará la evaluación de cuales son las principales variables para el modelo

In [108]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif


# Definir cuántas características queremos seleccionar
k = 7  

# Aplicar SelectKBest con f_classif (ANOVA F-score)
selector = SelectKBest(score_func=f_classif, k=k)
X_selected = selector.fit_transform(df_inicial_numeric, target)

# Obtener los puntajes de cada característica
scores = selector.scores_

# Crear un DataFrame con los resultados
feature_scores = pd.DataFrame({
    'Feature': df_inicial_numeric.columns,
    'Score': scores
}).sort_values(by='Score', ascending=False)

# Imprimir los resultados
print("Puntajes de características numéricas:")
print(feature_scores)


Puntajes de características numéricas:
                              Feature         Score
0                           Elevation  31087.079657
5     Horizontal_Distance_To_Roadways   2050.314700
2                               Slope   1559.369722
9  Horizontal_Distance_To_Fire_Points   1452.737911
6                       Hillshade_9am    643.862634
7                      Hillshade_Noon    595.275348
3    Horizontal_Distance_To_Hydrology    488.221292
4      Vertical_Distance_To_Hydrology    241.029561
8                       Hillshade_3pm    190.497164
1                              Aspect     88.987497


Revisando los score resultantes y viendo cuales fueron las variables seleccionadas, se toma la decisión de eliminar las variables Vertical_Distance_To_Hydrology, Hillshade_3pm y Aspect, dado que no aportan al modelo

In [109]:
selected_features = df_inicial_numeric.columns[selector.get_support()]
print("Variables numéricas seleccionadas:", selected_features.tolist())


Variables numéricas seleccionadas: ['Elevation', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Horizontal_Distance_To_Fire_Points']


In [110]:
df_inicial_numeric=df_inicial_numeric.drop(columns=["Vertical_Distance_To_Hydrology", "Hillshade_3pm" , "Aspect"])

df_variables=pd.concat([df_inicial_numeric, df_inicial_categoric, target ],axis=1)
df_variables


,Elevation,Slope,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,7,67,1015,233,234,1570,Commanche,C7202,1
1,2876,18,485,2495,192,202,1557,Commanche,C7757,1
2,3171,2,277,4374,213,237,1052,Rawah,C7745,0
3,3087,13,190,4774,193,221,752,Rawah,C7745,0
4,2835,10,212,3596,231,242,3280,Rawah,C4744,1
...,...,...,...,...,...,...,...,...,...,...
116198,3150,16,285,2275,200,253,866,Commanche,C7756,1
116199,3125,13,234,2430,224,212,1426,Rawah,C7745,0
116200,3166,11,67,1275,234,240,2404,Rawah,C7202,0
116201,3154,14,738,6012,181,239,1320,Rawah,C7745,1


# 3. Data Pipeline

## 3.1 Configurar el contexto interactivo

Se realiza la carga de las librerías necesarias para usar TFX

In [111]:
import tensorflow as tf

# Importar componentes de TFX
from tfx.components import CsvExampleGen, ExampleValidator, SchemaGen, StatisticsGen, Transform
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

Se configura el contexto interactivo, adicionalmente se indica la ruta en donde se guardarán los artefactos del pipeline

In [112]:
_pipeline_root = './pipeline/'

context = InteractiveContext(pipeline_root=_pipeline_root)

## 3.2. Generando Ejemplos

primero debemos guardar nuestra información en un archivo csv que pueda ser utilizado por TFX

In [113]:
# Definir la ruta donde se guardará el archivo CSV
data_dir = os.path.join(os.getcwd(), "data_tfx")
os.makedirs(data_dir, exist_ok=True)

csv_path = os.path.join(data_dir, "data.csv")

# Guardar el DataFrame como CSV (sin el índice para evitar problemas)
df_variables.to_csv(csv_path, index=False)

Se realiza la transformación de la información al formato requerido por TFX, adicionalmente se ejecuta el componente dentro del pipeline interactivo

In [114]:
example_gen = CsvExampleGen(input_base=data_dir)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Se obtiene el artefacto, en donde se verifica los nombres de los conjuntos en los que se ha dividido la data, adicionalmente la ruta donde se está guardando el pipeline inicial

In [115]:
artifact = example_gen.outputs['examples'].get()[0]

print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: ./pipeline/CsvExampleGen/examples/1


Se verifica la ruta donde se tienen los datos de entrenamiento y el nombre del archivo con la data generada

In [116]:
train_uri = os.path.join(artifact.uri, 'Split-train')

!ls {train_uri}

data_tfrecord-00000-of-00001.gz


Se cargan los ejemplos de entrenamiento en un dataset

In [117]:
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

Se define una función que permite visualizar los n primeros registros del dataset seleccionado, posteriormente se imprimen los 3 primeros datos

In [118]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

sample_records = get_records(dataset, 3)
pp.pprint(sample_records)    

[{'features': {'feature': {'Cover_Type': {'int64List': {'value': ['1']}},
                           'Elevation': {'int64List': {'value': ['2876']}},
                           'Hillshade_9am': {'int64List': {'value': ['192']}},
                           'Hillshade_Noon': {'int64List': {'value': ['202']}},
                           'Horizontal_Distance_To_Fire_Points': {'int64List': {'value': ['1557']}},
                           'Horizontal_Distance_To_Hydrology': {'int64List': {'value': ['485']}},
                           'Horizontal_Distance_To_Roadways': {'int64List': {'value': ['2495']}},
                           'Slope': {'int64List': {'value': ['18']}},
                           'Soil_Type': {'bytesList': {'value': ['Qzc3NTc=']}},
                           'Wilderness_Area': {'bytesList': {'value': ['Q29tbWFuY2hl']}}}}},
 {'features': {'feature': {'Cover_Type': {'int64List': {'value': ['0']}},
                           'Elevation': {'int64List': {'value': ['3171']}},
 

2025-02-26 02:30:05.051824: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## 3.3. Estadísticas

A partir del conjunto de ejemplo generado se ejecutaran las estadisticas tanto para el conjunto de entrenamiento como para el de validación

In [119]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Ahora veremos las estadísticas generadas para ambos conjuntos

In [120]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

## 3.4. Inferir el esquema

Para poder inferir el esquema, primero debemos isntanciarlo y ejecutarlo

In [121]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Ahora revisando el esquema inferido, vemos que cumple con lo esperado, tenemos 2 variables categoricas y el resto de variables son númericas

In [122]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## 3.5. Curando el esquema

In [123]:
import tensorflow_data_validation as tfdv
from tensorflow_metadata.proto.v0 import schema_pb2

# Obtener la ruta del esquema
schema_uri = schema_gen.outputs['schema'].get()[0].uri

# Cargar el esquema en memoria
schema = tfdv.load_schema_text(schema_uri + "/schema.pbtxt")


In [124]:
# Definir un dominio para 'Elevation'
Hillshade_9am_domain = schema_pb2.FloatDomain(
    name='Hillshade_9am_domain',min=0.0,max=255.0 )
tfdv.set_domain(schema, 'Hillshade_9am', Hillshade_9am_domain)


Hillshade_Noon_domain = schema_pb2.FloatDomain(
    name='Hillshade_Noon_domain',min=0.0,max=255.0 )
tfdv.set_domain(schema, 'Hillshade_Noon', Hillshade_Noon_domain)

slope_domain = schema_pb2.FloatDomain(
    name='slope_domain', min=0.0, max=90.0
)
tfdv.set_domain(schema, 'Slope', slope_domain)


In [125]:
cover_type_domain = schema_pb2.StringDomain(
    name="cover_type_domain",
    value=['0', '1', '2', '3', '4', '5', '6']  # Clases permitidas
)

# Asignar el dominio a Cover_Type con tfdv.set_domain()
tfdv.set_domain(schema, 'Cover_Type', cover_type_domain)

# Cambiar Cover_Type a CATEGÓRICO (BYTES en TFDV)
for feature in schema.feature:
    if feature.name == "Cover_Type":
        feature.type = schema_pb2.FeatureType.BYTES  # Convertimos a BYTES para que sea categórico
        feature.domain = "cover_type_domain"  # Asignamos el dominio categórico


In [126]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',STRING,required,,'cover_type_domain'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0.000000; max: 255.000000
'Hillshade_Noon',INT,required,,min: 0.000000; max: 255.000000
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0.000000; max: 90.000000
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [127]:
tfdv.write_schema_text(schema, schema_uri + "/schema.pbtxt")

## 3.6. Entornos de esquema

## 3.7. Genere nuevas estadísticas usando el esquema actualizado

## 3.8. Comprobar anomalías

## 3.9. Ingeniería de características

## 3.10. Función de preprocesamiento

## 3.11. Transformar